In [2]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-noisy-translation-b.cari.com.my/resolve/main/processed-b.cari.com.my.jsonl

In [3]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [7]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    'mesolitica/translation-t5-small-standard-bahasa-cased-v2',
    use_fast=False
)
model = T5ForConditionalGeneration.from_pretrained(
    'mesolitica/translation-t5-small-standard-bahasa-cased-v2'
)
all_special_ids = [0, 1, 2]

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
_ = model.cuda()

In [1]:
import json
import re
import torch
from tqdm import tqdm

def clean(string):
    string = re.sub(
        'http\\S+|www.\\S+',
        '',
        ' '.join(
            [
                word
                for word in string.split()
                if word.find('#') < 0 and word.find('@') < 0
            ]
        ),
    )
    return string

In [2]:
ls = []
with open('processed-b.cari.com.my.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        l['cleaned'] = clean(l['left'])
        ls.append(l)

631727it [00:04, 142827.45it/s]


In [3]:
ls[0]

{'left': 'bagusla mawi bagi peluang kat junior2 dia yang berbakat tak kira lelaki atau perempuan untuk tonjolkan bakat, bila appear dgn mawi ada la orang tertarik nak ambik lagi',
 'en': "It's good that Mawi gives opportunities to talented juniors regardless of gender to showcase their talents. When they appear with Mawi, there are people who are interested in taking them on again.",
 'ms': 'Baguslah Mawi memberi peluang kepada junior-junior yang berbakat tanpa mengira jantina untuk menonjolkan bakat mereka. Apabila mereka muncul bersama Mawi, terdapat orang yang berminat untuk mengambil mereka lagi.',
 'cleaned': 'bagusla mawi bagi peluang kat junior2 dia yang berbakat tak kira lelaki atau perempuan untuk tonjolkan bakat, bila appear dgn mawi ada la orang tertarik nak ambik lagi'}

In [12]:
# %%time

# s = 'Dalam hujan lebat some more'
# input_ids = tokenizer.encode(f'terjemah ke pasar Melayu: {s}', return_tensors = 'pt')
# outputs = model.generate(input_ids.cuda(), max_length = 100, do_sample=True,
#     top_k=50,
#     top_p=0.95,
#     num_return_sequences=5, temperature = 1.0, output_scores = True, return_dict_in_generate = True)
# seqs = []
# for o in outputs.sequences:
#     o = tokenizer.decode([i for i in o if i not in all_special_ids], 
#                          spaces_between_special_tokens = False)
#     seqs.append(o)
# seqs

In [13]:
# !rm -rf b.cari.com.my-predict
!mkdir b.cari.com.my-predict

In [14]:
pairs = {
    'en': 'Manglish',
    'ms': 'pasar Melayu'
}

In [ ]:
for i in tqdm(range(len(ls))):
    filename = os.path.join('b.cari.com.my-predict', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    results = {'original': ls[i]}
        
    for lang, prefix in pairs.items():
    
        if ls[i][lang] and len(ls[i][lang]) > 5:
            s = ls[i][lang]
            input_ids = tokenizer.encode(f'terjemah ke {prefix}: {s}', return_tensors = 'pt')
            outputs = model.generate(input_ids.cuda(), max_length = 256, do_sample=True,
                top_k=50,
                top_p=0.95,
                num_return_sequences=5, temperature = 0.7, output_scores = True, return_dict_in_generate = True)
            logits = torch.stack(outputs.scores, dim=1)
            score = logits.max(dim = -1).values.mean(dim = -1).detach().cpu().numpy().tolist()
            seqs = []
            for o in outputs.sequences:
                o = tokenizer.decode([i for i in o if i not in all_special_ids], 
                                     spaces_between_special_tokens = False)
                seqs.append(o)
            
            results[lang] = {
                'score': score,
                'sequences': seqs
            }
        
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)

 12%|██████████                                                                            | 73602/631727 [18:17:54<55:19:31,  2.80it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 15%|█████████████▏                                                                        | 96486/631727 [24:00:35<48:49:28,  3.05it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 18%|███████████████                                                          

In [4]:
from glob import glob
import json

In [5]:
files = glob('b.cari.com.my-predict/*.json')
len(files)

125517

In [6]:
predicted = []
for f in tqdm(files):
    try:
        with open(f) as fopen:
            l = json.load(fopen)
        predicted.append(l)
    except:
        pass

100%|████████████████████████████████████████████████████████████████████████████████████████| 125517/125517 [00:02<00:00, 49503.23it/s]


In [7]:
predicted[0]

{'original': {'left': 'jangan pakai hub.. kalu boleh pakai switch.. lebih laju..  hub ader jual lagi ker skarang ni? mcm dah takder .. paling minimum pun 5 port switch',
  'en': "Don't use a hub, if possible use a switch, it's faster. Are hubs still being sold now? It seems like they're not available anymore. At the very least, a switch should have a minimum of 5 ports.",
  'ms': 'Jangan gunakan hub, jika boleh gunakan switch, ia lebih pantas. Adakah hub masih dijual sekarang? Nampaknya sudah tiada lagi. Sekurang-kurangnya, satu switch perlu mempunyai 5 port.',
  'cleaned': 'jangan pakai hub.. kalu boleh pakai switch.. lebih laju.. hub ader jual lagi ker skarang ni? mcm dah takder .. paling minimum pun 5 port switch'},
 'en': {'score': [32.21791076660156,
   27.90170669555664,
   27.39642333984375,
   30.034753799438477,
   27.076414108276367],
  'sequences': ["Don't use hub, if possible use switch faster. Are hubs still being sold now? Looks like they are no more. At the very least a 

In [8]:
with open('noisy-augmentation-b.cari.com.my.jsonl', 'w') as fopen:
    for p in tqdm(predicted):
        fopen.write(f'{json.dumps(p)}\n')

100%|████████████████████████████████████████████████████████████████████████████████████████| 125517/125517 [00:01<00:00, 94741.06it/s]


In [9]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='noisy-augmentation-b.cari.com.my.jsonl',
    path_in_repo='noisy-augmentation-b.cari.com.my.jsonl',
    repo_id='mesolitica/noisy-augmentation',
    repo_type='dataset',
)

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


noisy-augmentation-b.cari.com.my.jsonl:   0%|          | 0.00/322M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/noisy-augmentation/commit/74be23b8780098b8f67fb5e10af0cb95767df4e6', commit_message='Upload noisy-augmentation-b.cari.com.my.jsonl with huggingface_hub', commit_description='', oid='74be23b8780098b8f67fb5e10af0cb95767df4e6', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
ranged = range(len(ls))

In [11]:
def overlap(string1, string2):
    l = set([w for w in clean(string1).split() if len(w) > 2])
    r = set([w for w in clean(string2).split() if len(w) > 2])
    return len(l & r) / len(l)

overlap(ls[0]['left'], ls[1]['left'])

0.0

In [12]:
import random

sampled = random.sample(ranged, 100)
negs = []
for s in sampled:
    overlapped = overlap(ls[0]['ms'], ls[s]['ms'])
    if overlapped < 0.05:
        negs.append(ls[s]['left'])
    if len(negs) >= 5:
        break
        
negs

['Haaaa proper sikit. Even hasil curi;p  Haha.. iols tak redha curi2 nih..',
 'check in..nih apa?? rumah ker pondok kayu???  comeynyer!!!~',
 'CUma patut ada verse khas utk gitar solo Slash.',
 'TERmoneng..',
 'same goes to malaysia yg mane anak2 bangla dan indon akan makin ramai']

In [13]:
!rm -rf mining-b.cari.com.my
!mkdir mining-b.cari.com.my

In [14]:
import os

def loop(rows):
    rows, index = rows
    for i in tqdm(range(len(rows))):
        filename = os.path.join('mining-b.cari.com.my', f'{i}-{index}.json')
        if os.path.exists(filename):
            continue
        
        sampled = random.sample(ranged, 100)
        negs = []
        for s in sampled:
            try:
                overlapped = overlap(rows[i]['original']['ms'], ls[s]['ms'])
            except:
                continue
            if overlapped < 0.05:
                negs.append(ls[s]['left'])
            if len(negs) >= 5:
                break
                
        en = rows[i]['original']['en']
        ms = rows[i]['original']['ms']
        
        en_augmentation = []
        try:
            for no, score in enumerate(rows[i]['en']['score']):
                if score > 30:
                    en_augmentation.append(rows[i]['en']['sequences'][no])
            en_augmentation = list(set(en_augmentation))
        except:
            pass
        
        ms_augmentation = []
        try:
            for no, score in enumerate(rows[i]['ms']['score']):
                if score > 30:
                    ms_augmentation.append(rows[i]['ms']['sequences'][no])
            ms_augmentation = list(set(ms_augmentation))
        except:
            pass
        
        try:
            d = {
                'negs': negs,
                'pos': list(set([en, ms] + en_augmentation + ms_augmentation)),
                'query': rows[i]['original']['left'],
            }

            with open(filename, 'w') as fopen:
                json.dump(d, fopen)
        except:
            pass

In [15]:
loop((predicted[:1000],0))

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 4609.28it/s]


In [16]:
import mp

In [17]:
mp.multiprocessing(predicted, loop, cores = 10, returned = False)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 12551/12551 [00:02<00:00, 4597.56it/s]


In [18]:
files = glob('mining-b.cari.com.my/*.json')
len(files)

125517

In [19]:
with open('mining-b.cari.com.my.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
            fopen_l.write(f'{json.dumps(data)}\n')
            fopen_l.flush()
        except:
            pass

100%|████████████████████████████████████████████████████████████████████████████████████████| 125517/125517 [00:02<00:00, 57616.98it/s]


In [20]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='mining-b.cari.com.my.jsonl',
    path_in_repo='mining-b.cari.com.my.jsonl',
    repo_id='mesolitica/title-context-pair',
    repo_type='dataset',
)

mining-b.cari.com.my.jsonl:   0%|          | 0.00/245M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/title-context-pair/commit/c4ec90de6434a3e6742342219a4b70716f83d342', commit_message='Upload mining-b.cari.com.my.jsonl with huggingface_hub', commit_description='', oid='c4ec90de6434a3e6742342219a4b70716f83d342', pr_url=None, pr_revision=None, pr_num=None)